# Azure AI Content Understanding

Azure AI Content Understanding is a new generative AI-based Azure AI Service that analyzes files of any modality (documents, images, videos, and audio) and extracts structured output in user-defined field formats.

Requirements:
1. Azure AI Services

Note: Check supported regions: https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/language-region-support?tabs=document#region-support

## Load Azure Configuration

In [25]:
import os

azure_ai_services_endpoint = os.environ["AZURE_AI_SERVICES_ENDPOINT"]
azure_ai_services_key = os.environ["AZURE_AI_SERVICES_KEY"]

## Create a Custom Analyzer

In [26]:
import requests
import json

# Create a new analyzer
def create_analyzer(analyzer_id, request_body):
    endpoint = azure_ai_services_endpoint
    key = azure_ai_services_key
    
    url = f"{endpoint}/contentunderstanding/analyzers/{analyzer_id}?api-version=2024-12-01-preview"
    headers = {
        "Ocp-Apim-Subscription-Key": key,
        "Content-Type": "application/json"
    }
    
    response = requests.put(url, headers=headers, data=json.dumps(request_body))
    # The 201 (Created) response includes an Operation-Location header containing a URL that you can use to track the status of this asynchronous creation operation.
    operation_location = response.headers.get("Operation-Location")
    return operation_location

In [27]:
ANALYZER_NAME = "ImageAnalyzer"

request_body = {
    "description": "Sample Image analyzer",
    "scenario": "image",
    "fieldSchema": {
        "fields": {
            "title": {
                "type": "string"
            },
            "imagedescription": {
                "type": "string",
                "method": "generate"
            },
            "imagecategory": {
                "type": "string",
                "method": "classify",
                "enum": ["car", "bus", "bicycle"]
            }
        }
    }
}

custom_analyzer_operation_location = create_analyzer(ANALYZER_NAME, request_body)

print(custom_analyzer_operation_location)

https://ziggyazureaiserviceswestus.services.ai.azure.com/contentunderstanding/analyzers/ImageAnalyzer/operations/6cdebf4b-8423-435c-8b0b-4eb69bf00279?api-version=2024-12-01-preview


## Track Status of Asynchronous Create Operation

In [28]:
import requests

def get_analyzer_operation_status(url):
    
    key = azure_ai_services_key
    
    headers = {
        "Ocp-Apim-Subscription-Key": key
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        print("Operation status retrieved successfully.")
        print(response.json())
    else:
        print(f"Failed to retrieve operation status. Status code: {response.status_code}, Response: {response.text}")

In [29]:
response = get_analyzer_operation_status(custom_analyzer_operation_location)

Operation status retrieved successfully.
{'id': '6cdebf4b-8423-435c-8b0b-4eb69bf00279', 'status': 'Succeeded', 'result': {'analyzerId': 'ImageAnalyzer', 'description': 'Sample Image analyzer', 'createdAt': '2025-03-17T08:53:30Z', 'lastModifiedAt': '2025-03-17T08:53:30Z', 'config': {'returnDetails': False, 'disableContentFiltering': False}, 'fieldSchema': {'fields': {'title': {'type': 'string'}, 'imagedescription': {'type': 'string', 'method': 'generate'}, 'imagecategory': {'type': 'string', 'method': 'classify', 'enum': ['car', 'bus', 'bicycle']}}}, 'warnings': [], 'status': 'ready', 'scenario': 'image'}}


## Analyze a file

In [30]:
import requests

def analyze_file(analyzer_id, file_url):
    
    endpoint = azure_ai_services_endpoint
    key = azure_ai_services_key

    api_version = "2024-12-01-preview"
    url = f"{endpoint}/contentunderstanding/analyzers/{analyzer_id}:analyze?api-version={api_version}"
    headers = {
        "Ocp-Apim-Subscription-Key": key,
        "Content-Type": "application/json"
    }
    data = {
        "url": file_url
    }

    response = requests.post(url, headers=headers, json=data)
    
    operation_location = response.headers.get("Operation-Location")
    # print(f"Operation-Location: {operation_location}")
    return operation_location


In [31]:
analyze_file_operation_location = analyze_file(ANALYZER_NAME, "https://ziggystorage01.blob.core.windows.net/images/bus1.jpeg")
print(analyze_file_operation_location)

https://ziggyazureaiserviceswestus.services.ai.azure.com/contentunderstanding/analyzers/ImageAnalyzer/results/3e144865-aa2f-40f1-a5dc-90adad3eb829?api-version=2024-12-01-preview


## Track Status of Analysis

In [32]:
import requests

def analyze_file_track_status(url):
    key = azure_ai_services_key

    headers = {
        "Ocp-Apim-Subscription-Key": key,
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        response_json = response.json()
        status = response_json.get("status")
        if status == "Succeeded":
            return response_json
        else:
            return status
    else:
        response.raise_for_status()

In [33]:
result = analyze_file_track_status(analyze_file_operation_location)

if isinstance(result, dict) and result.get("status") == "Succeeded":
    contents = result.get("result", {}).get("contents", [])
    if contents:
        fields = contents[0].get("fields", {})
        image_description = fields.get("imagedescription", {}).get("valueString")
        image_category = fields.get("imagecategory", {}).get("valueString")
        title = fields.get("title", {}).get("valueString")
        
        print(f"Title: {title}")
        print(f"Image Description: {image_description}")
        print(f"Image Category: {image_category}")
else:
    print(result)

Title: Yellow Bus
Image Description: The image shows a yellow bus with a modern design. It has a large windshield and side windows, providing ample visibility for passengers. The bus is positioned at an angle, showcasing its front and side. It has a sleek, aerodynamic shape with a smooth, rounded front and a rectangular body. The bus is likely used for public transportation, given its size and design.
Image Category: bus


In [34]:
analyze_file_operation_location = analyze_file(ANALYZER_NAME, "https://ziggystorage01.blob.core.windows.net/images/car1.jpeg")

In [38]:
result = analyze_file_track_status(analyze_file_operation_location)

if isinstance(result, dict) and result.get("status") == "Succeeded":
    contents = result.get("result", {}).get("contents", [])
    if contents:
        fields = contents[0].get("fields", {})
        image_description = fields.get("imagedescription", {}).get("valueString")
        image_category = fields.get("imagecategory", {}).get("valueString")
        title = fields.get("title", {}).get("valueString")
        
        print(f"Title: {title}")
        print(f"Image Description: {image_description}")
        print(f"Image Category: {image_category}")
else:
    print(result)

Title: Image of a Car
Image Description: The image shows a silver-colored hatchback car. It has a sleek design with a modern front grille and stylish headlights. The car is positioned at an angle, showcasing its front and side profile. It has five doors, including the rear hatch, and is equipped with alloy wheels. The windows are tinted, and the car appears to be in a studio setting with a white background.
Image Category: car


## Delete Analyzer

In [39]:
import requests

def delete_analyzer(analyzer_id):
    endpoint = azure_ai_services_endpoint
    key = azure_ai_services_key

    api_version = "2024-12-01-preview"
    url = f"{endpoint}/contentunderstanding/analyzers/{analyzer_id}?api-version={api_version}"
    
    headers = {
        "Ocp-Apim-Subscription-Key": key,
        "Content-Type": "application/json"
    }

    response = requests.delete(url, headers=headers)
    
    if response.status_code == 204:
        return "Analyzer deleted successfully."
    else:
        response.raise_for_status()

In [51]:
result = delete_analyzer(ANALYZER_NAME)
print(result)

Analyzer deleted successfully.


## Analyzing Video Example

In [52]:
ANALYZER_NAME = "VideoAnalyzerZZ"

request_body = {
    "description": "Sample Video analyzer",
    "scenario": "videoShot",
    "fieldSchema": {
        "fields": {
            "title": {
                "type": "string"
            },
            "videodescription": {
                "type": "string",
                "method": "generate",
                "description": "Detailed summary of the video segment"
            },
            "company": {
                "type": "string",
                "method": "generate",
                "description": "The company that this video segment is about"
            },
            "azure": {
                "type": "string",
                "method": "generate",
                "description": "The Microsoft Azure service used by the company in this video"
            },
            "azurepurpose": {
                "type": "string",
                "method": "generate",
                "description": "The purpose of why the Microsoft Azure service is being used by the company"
            },
        }
    }
}

custom_analyzer_operation_location = create_analyzer(ANALYZER_NAME, request_body)

print(custom_analyzer_operation_location)

https://ziggyazureaiserviceswestus.services.ai.azure.com/contentunderstanding/analyzers/VideoAnalyzerZZ/operations/c3da6e5e-f5b5-4e18-b1fe-6c74edae90a0?api-version=2024-12-01-preview


In [53]:
response = get_analyzer_operation_status(custom_analyzer_operation_location)

Operation status retrieved successfully.
{'id': 'c3da6e5e-f5b5-4e18-b1fe-6c74edae90a0', 'status': 'Succeeded', 'result': {'analyzerId': 'VideoAnalyzerZZ', 'description': 'Sample Video analyzer', 'createdAt': '2025-03-17T08:59:31Z', 'lastModifiedAt': '2025-03-17T08:59:31Z', 'config': {'locales': ['en-US', 'es-ES', 'es-MX', 'fr-FR', 'hi-IN', 'it-IT', 'ja-JP', 'ko-KR', 'pt-BR', 'zh-CN'], 'returnDetails': False, 'enableFace': False, 'disableContentFiltering': False}, 'fieldSchema': {'fields': {'title': {'type': 'string'}, 'videodescription': {'type': 'string', 'method': 'generate', 'description': 'Detailed summary of the video segment'}, 'company': {'type': 'string', 'method': 'generate', 'description': 'The company that this video segment is about'}, 'azure': {'type': 'string', 'method': 'generate', 'description': 'The Microsoft Azure service used by the company in this video'}, 'azurepurpose': {'type': 'string', 'method': 'generate', 'description': 'The purpose of why the Microsoft Azure

In [57]:
analyze_file_operation_location = analyze_file(ANALYZER_NAME, "https://ziggystorage01.blob.core.windows.net/videos/mercedes.mp4")
print(analyze_file_operation_location)

https://ziggyazureaiserviceswestus.services.ai.azure.com/contentunderstanding/analyzers/VideoAnalyzerZZ/results/7d3c5028-afc5-4e0e-8575-f8866f6a3945?api-version=2024-12-01-preview


In [83]:
result = analyze_file_track_status(analyze_file_operation_location)

if isinstance(result, dict) and result.get("status") == "Succeeded":
    contents = result.get("result", {}).get("contents", [])
    if contents:
        fields = contents[0].get("fields", {})
        title = fields.get("title", {}).get("valueString")
        company = fields.get("company", {}).get("valueString")
        video_description = fields.get("videodescription", {}).get("valueString")
        azure = fields.get("azure", {}).get("valueString")
        azurepurpose = fields.get("azurepurpose", {}).get("valueString")

        print(f"Title: {title}")
        print(f"Company: {company}")
        print(f"Video Description: {video_description}")
        print(f"Azure Product: {azure}")
        print(f"Azure Purpose: {azurepurpose}")
        
else:
    print(result)

Title: Mercedes-Benz Car on Scenic Drive
Company: Mercedes-Benz
Video Description: A Mercedes-Benz car is driving along a scenic road with mountains in the background. The video highlights the vehicle's sleek design and dynamic motion as it navigates the curved road.
Azure Product: Azure AI
Azure Purpose: Azure AI is used to enhance the user experience through personalized interactions and vehicle adaptability.


In [33]:
result = delete_analyzer(ANALYZER_NAME)
print(result)

Analyzer deleted successfully.
